In [32]:
import pandas as pd
import numpy as np
import re
import openpyxl
import datetime as dt
import sqlite3
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
import decimal

import os
import sys
# SRC_DIR = r'/home/pp/program/jupyter/'
sys.path.append(os.path.dirname(r'/home/pp/program/jupyter/'))
from config import *

# Replace 'your_file.parquet' with the path to your Parquet file
df = pd.read_parquet(TEPLOTY_SK_DIR + 'teploty_sk.parquet')
tf =df.copy()

print(df.head())

                Stanica  Teplota Smer  Rýchlosť Nárazy    Tlak Oblačnosť  \
49       Oravské Veselé     20.9    Z       2.0   None     NaN      None   
25               Košice     22.8   SZ       3.0   None  1013.6   Oblačno   
26  Košice - Podhradová     24.1   SV       2.0   None  1013.2      None   
27    Kráľova pri Senci     24.3    Z       3.0   None     NaN      None   
28       Kremnické Bane     20.2   SZ       2.0   None     NaN      None   

   Počasie  Graf             Cas_CET              file  
49    None   NaN 2025-07-30 14:00:00  2025-07-30-15-00  
25    None   NaN 2025-07-30 14:00:00  2025-07-30-15-00  
26    None   NaN 2025-07-30 14:00:00  2025-07-30-15-00  
27    None   NaN 2025-07-30 14:00:00  2025-07-30-15-00  
28    None   NaN 2025-07-30 14:00:00  2025-07-30-15-00  


In [36]:
dfbrez = df[df.Stanica == 'Brezno']
maskbrez = df.Stanica == 'Brezno'

dfbrez = df.loc[maskbrez,['Stanica', 'Smer', 'Rýchlosť', 'Cas_CET']] #OK
dfbrez = df.loc[df.Stanica == 'Brezno',['Stanica', 'Smer', 'Rýchlosť', 'Cas_CET']] #OK

directions = {'S': 0, 'SV': 45, 'V': 90, 'JV' : 135, 'J' : 180, 'JZ' : 225,  'Z' : 270, 'SZ' : 315, 'BEZV' : 400, None : 500}
directions['J']


180

In [39]:
def k(x):
    return directions[x]
dfbrez
k('SV')
dfbrez['winddir'] = dfbrez.Smer.apply(lambda x: directions[x])
dfbrez.info


<bound method DataFrame.info of    Stanica  Smer  Rýchlosť             Cas_CET  winddir
8   Brezno     Z       2.0 2025-07-30 14:00:00      270
8   Brezno    JZ       2.0 2025-07-30 15:00:00      225
8   Brezno    JZ       2.0 2025-07-30 15:00:00      225
8   Brezno    JZ       2.0 2025-07-30 15:00:00      225
8   Brezno    JZ       2.0 2025-07-30 15:00:00      225
..     ...   ...       ...                 ...      ...
8   Brezno     V       1.0 2025-09-25 17:00:00       90
8   Brezno     V       1.0 2025-09-25 18:00:00       90
8   Brezno     V       1.0 2025-09-25 19:00:00       90
8   Brezno  BEZV       0.0 2025-09-25 20:00:00      400
8   Brezno    JV       1.0 2025-09-25 21:00:00      135

[1376 rows x 5 columns]>